<!-- PROJECT LOGO -->
<br/>
<p align="center">
    <img src="https://raw.githubusercontent.com/Team-17-Bedu/proyecto_python/main/img/icono.png" alt="Logo" width="135" height="135">

  <h3 align="center"><strong>Países con mayor calidad de vida</strong></h3>

  <p align="center">
    Proyecto final para el modulo "Procesamiento de datos con Python"
  </p>
</p>

#### Acerca del proyecto:

Este proyecto tiene como proposito realizar un analisis de la calidad de vida de ciertos paises, con el fin de determinar cuál de ellos podria ser la mejor opcion para vivir o buscar un empleo, todo esto, haciendo uso de herramientas estadísticas y computacionales, siendo el principal aliado, el lenguaje de programacion Python. 


En este caso se tendra como principal objetivo en calcular el IDH por medio de la documentación proporcionada del __*Human Development Report 2020*__, presente en el siguiente [enlace](http://hdr.undp.org/sites/default/files/hdr2020_technical_notes.pdf). Y con tales resultados justificar nuestro anterior proyecto elaborado en el lenguaje R, el proyecto esta presente en el siguiente [enlace](https://github.com/Team-17-Bedu/proyecto).

Todos los indicadores serán obtenidos por medio de la API proporcionada por la [UNDP](http://ec2-54-174-131-205.compute-1.amazonaws.com/API/Login.php), de los cuales serán analizados 
* Años esperados de escolaridad 
* Ingreso nacional bruto (INB) per cápita
* Esperanza de vida al nacer
* Promedio de años de escolaridad

<br/>
<p align="center">
    <img src="https://raw.githubusercontent.com/Team-17-Bedu/proyecto_python/main/img/logo-undp.jpg">
</p>

### Preparación:
Para comenzar se importaron los modulos necesarios (dadas las limitaciones de github no es posible renderizar los widgets, para ello sera necesario visualuzar el notebook en Google Colab)


In [ ]:
import requests
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display

Debido a que el acceso a la API requiere un inicio de sesion, se recurrió al objeto `Session` del modulo `requests`  

In [ ]:
session = requests.Session()

Mediante el modulo `widgets` se definieron dos text box para introducir los datos de autenticación, `user` y `password` respectivamente 

In [ ]:
user = widgets.Text()
password = widgets.Text()
display(user, password)

Text(value='')

Text(value='')

Definición de la tupla que contendra los datos de acceso

In [ ]:
access = (user.value, password.value)

Inicio de sesión en la API de la UNDP

In [ ]:
session.get("http://ec2-54-174-131-205.compute-1.amazonaws.com/API/Login.php",
            auth=access)

<Response [200]>

Puesto que el `json` recibido como respuesta presenta una estructura un tanto caotica, fue necesario crear un algoritmo que se encargara de normalizar los datos, a modo de obtener un `DataFrame` homogeneo, para ello se definio la funcion `normalize_data`

In [ ]:
def normalize_data(data_dirt: dict, indicator_id: str) -> dict:
    new_data = {
        'Country': list(data_dirt['country_name'].values())
    }
    countries = list(data_dirt['country_name'].keys())
    for year, year_data in data_dirt['indicator_value'].items():
        values = []
        last_key = ''
        for key, val in year_data[indicator_id].items():
            current_pos = countries.index(key)
            last_pos = countries.index(last_key) if last_key != '' else 0
            last_key = key
            zeros = current_pos - last_pos
            i = 1
            while i < zeros and zeros != 0:
                values.append(0)
                i += 1
            values.append(val)
        new_data[year] = values
    return  new_data

Se declara la función `get_data` que construira la url mediante un `fstring` para la petición, esto es porque la url para la peticion tiene una sintaxis diferente, posteriormente se solicitarán los datos y se harán pasar por el normalizado definido con anterioridad

In [ ]:
def get_data(indicator: str) -> dict :
    url = f'http://ec2-54-174-131-205.compute-1.amazonaws.com/API/HDRO_API.php/indicator_id={indicator}/structure=yic'
    return normalize_data(session.get(url).json(), indicator)

### Calculo del Indice de Educación
#### Expectativa de años de escolaridad
Obtencion de los registros del indicador correspondiente a la expectativa de años de escolaridad (`ID` = 69706)

In [ ]:
data = get_data('69706')

Creación del `DataFrame` con los datos normalizados y visualización de `head` y `tail`

In [ ]:
expectativa = pd.DataFrame(data)
expectativa

,Country,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Afghanistan,2.593,2.919,3.246,3.573,3.899,4.226,4.553,4.879,5.206,5.533,5.860,6.186,6.513,6.840,7.858,8.125,8.393,8.660,8.754,8.939,9.532,9.478,9.973,10.172,10.262,10.235,10.262,10.139,10.139,10.176
1,Angola,3.441,3.252,3.234,3.657,3.767,3.877,3.987,4.096,4.206,4.638,5.070,5.501,5.933,6.365,6.796,7.228,7.660,8.091,8.523,8.955,8.646,9.545,9.916,10.286,10.657,11.028,11.399,11.777,11.777,11.777
2,Albania,11.603,11.764,10.664,10.127,10.091,10.166,10.228,10.505,10.669,10.685,10.589,10.684,10.702,10.905,10.885,11.419,11.582,11.964,12.125,12.264,13.000,13.748,14.587,14.926,15.252,15.076,14.805,14.816,14.696,14.696
3,Andorra,10.799,10.799,10.799,10.799,10.799,10.799,10.799,10.799,10.799,10.799,10.799,10.799,11.037,10.943,10.792,10.826,11.183,11.183,11.672,11.672,11.672,11.672,13.524,13.139,13.495,13.140,13.300,13.046,13.300,13.300
4,United Arab Emirates,10.314,10.663,10.507,10.539,10.860,11.093,10.847,10.550,10.376,10.529,10.681,10.834,10.987,11.139,11.292,11.445,11.598,11.750,11.903,12.056,12.208,12.361,12.514,12.666,13.005,13.667,13.643,13.643,14.344,14.344
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,Samoa,11.655,11.699,11.744,11.788,11.832,11.877,11.921,11.832,12.010,12.126,12.003,12.031,12.060,12.089,12.390,12.369,12.378,12.381,12.233,12.484,12.859,12.508,12.380,12.458,12.535,12.433,12.431,12.520,12.520,12.730
189,Yemen,7.531,7.545,7.559,7.573,7.587,7.602,7.616,7.630,7.594,7.658,7.852,8.046,8.240,8.434,8.618,8.661,8.584,8.507,8.418,8.530,8.589,8.977,8.457,8.809,8.737,8.701,8.664,8.664,8.664,8.769
190,South Africa,11.371,11.946,12.315,12.684,12.752,13.038,13.022,13.007,12.992,12.976,12.961,12.946,12.931,12.915,12.900,12.885,12.869,12.854,12.839,12.824,12.808,12.793,12.872,13.147,13.406,13.765,13.668,13.668,13.668,13.791
191,Zambia,7.518,7.744,7.971,8.197,8.424,8.650,8.876,9.103,9.329,9.556,9.782,10.009,10.235,10.462,10.688,10.915,11.141,10.924,11.004,11.010,11.017,10.932,10.931,10.960,11.046,11.133,11.219,11.305,11.392,11.478


* Dimensiones del `DataFrame`
    * 193 filas
    * 31 columnas

In [ ]:
expectativa.shape

(193, 31)

* Nombre de las columnas

In [ ]:
expectativa.columns

Index(['Country', '1990', '1991', '1992', '1993', '1994', '1995', '1996',
       '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005',
       '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014',
       '2015', '2016', '2017', '2018', '2019'],
      dtype='object')

#### Calculo del indice de expectativa de años de escolaridad por paises

Se explica a continuación como se realizo el calculo del indice de expectativa de años de escolaaridad por pais.
Primeramente se presenta lo que es la formula que se uso:

$$\Huge\Huge\frac{\alpha - \theta}{\gamma - \theta}$$

Donde:

* $\alpha$ : Es la expectativa de años de escolaridad en el pais.
* $\theta$ : Es la expectativa minima de años de escolaridad. En el documento __*Human Development Report 2020*__ se especifican 0 años.
* $\gamma$ : Es la expectativa maxima de años de escolaridad. En el documento __*Human Development Report 2020*__ se especifican 18 años.


Este calculo se realizo por cada pais en cada uno de los años.

In [ ]:
for year in range(1990, 2020):
    year = str(year)
    indices = []
    for cell in expectativa[year]:
        try:
            indices.append((float(cell) - 0) / (18 - 0))
        except:
            indices.append(0)
    expectativa[f'Indice_Expect_Educacion_{year}'] = indices
 
expectativa = expectativa.drop([str(i) for i in range(1990, 2020)],axis=1)

In [ ]:
expectativa

,Country,Indice_Expect_Educacion_1990,Indice_Expect_Educacion_1991,Indice_Expect_Educacion_1992,Indice_Expect_Educacion_1993,Indice_Expect_Educacion_1994,Indice_Expect_Educacion_1995,Indice_Expect_Educacion_1996,Indice_Expect_Educacion_1997,Indice_Expect_Educacion_1998,Indice_Expect_Educacion_1999,Indice_Expect_Educacion_2000,Indice_Expect_Educacion_2001,Indice_Expect_Educacion_2002,Indice_Expect_Educacion_2003,Indice_Expect_Educacion_2004,Indice_Expect_Educacion_2005,Indice_Expect_Educacion_2006,Indice_Expect_Educacion_2007,Indice_Expect_Educacion_2008,Indice_Expect_Educacion_2009,Indice_Expect_Educacion_2010,Indice_Expect_Educacion_2011,Indice_Expect_Educacion_2012,Indice_Expect_Educacion_2013,Indice_Expect_Educacion_2014,Indice_Expect_Educacion_2015,Indice_Expect_Educacion_2016,Indice_Expect_Educacion_2017,Indice_Expect_Educacion_2018,Indice_Expect_Educacion_2019
0,Afghanistan,0.144056,0.162167,0.180333,0.198500,0.216611,0.234778,0.252944,0.271056,0.289222,0.307389,0.325556,0.343667,0.361833,0.380000,0.436556,0.451389,0.466278,0.481111,0.486333,0.496611,0.529556,0.526556,0.554056,0.565111,0.570111,0.568611,0.570111,0.563278,0.563278,0.565333
1,Angola,0.191167,0.180667,0.179667,0.203167,0.209278,0.215389,0.221500,0.227556,0.233667,0.257667,0.281667,0.305611,0.329611,0.353611,0.377556,0.401556,0.425556,0.449500,0.473500,0.497500,0.480333,0.530278,0.550889,0.571444,0.592056,0.612667,0.633278,0.654278,0.654278,0.654278
2,Albania,0.644611,0.653556,0.592444,0.562611,0.560611,0.564778,0.568222,0.583611,0.592722,0.593611,0.588278,0.593556,0.594556,0.605833,0.604722,0.634389,0.643444,0.664667,0.673611,0.681333,0.722222,0.763778,0.810389,0.829222,0.847333,0.837556,0.822500,0.823111,0.816444,0.816444
3,Andorra,0.599944,0.599944,0.599944,0.599944,0.599944,0.599944,0.599944,0.599944,0.599944,0.599944,0.599944,0.599944,0.613167,0.607944,0.599556,0.601444,0.621278,0.621278,0.648444,0.648444,0.648444,0.648444,0.751333,0.729944,0.749722,0.730000,0.738889,0.724778,0.738889,0.738889
4,United Arab Emirates,0.573000,0.592389,0.583722,0.585500,0.603333,0.616278,0.602611,0.586111,0.576444,0.584944,0.593389,0.601889,0.610389,0.618833,0.627333,0.635833,0.644333,0.652778,0.661278,0.669778,0.678222,0.686722,0.695222,0.703667,0.722500,0.759278,0.757944,0.757944,0.796889,0.796889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,Samoa,0.647500,0.649944,0.652444,0.654889,0.657333,0.659833,0.662278,0.657333,0.667222,0.673667,0.666833,0.668389,0.670000,0.671611,0.688333,0.687167,0.687667,0.687833,0.679611,0.693556,0.714389,0.694889,0.687778,0.692111,0.696389,0.690722,0.690611,0.695556,0.695556,0.707222
189,Yemen,0.418389,0.419167,0.419944,0.420722,0.421500,0.422333,0.423111,0.423889,0.421889,0.425444,0.436222,0.447000,0.457778,0.468556,0.478778,0.481167,0.476889,0.472611,0.467667,0.473889,0.477167,0.498722,0.469833,0.489389,0.485389,0.483389,0.481333,0.481333,0.481333,0.487167
190,South Africa,0.631722,0.663667,0.684167,0.704667,0.708444,0.724333,0.723444,0.722611,0.721778,0.720889,0.720056,0.719222,0.718389,0.717500,0.716667,0.715833,0.714944,0.714111,0.713278,0.712444,0.711556,0.710722,0.715111,0.730389,0.744778,0.764722,0.759333,0.759333,0.759333,0.766167
191,Zambia,0.417667,0.430222,0.442833,0.455389,0.468000,0.480556,0.493111,0.505722,0.518278,0.530889,0.543444,0.556056,0.568611,0.581222,0.593778,0.606389,0.618944,0.606889,0.611333,0.611667,0.612056,0.607333,0.607278,0.608889,0.613667,0.618500,0.623278,0.628056,0.632889,0.637667


#### Expectativa de años de escolaridad
Obtencion de los registros del indicador correspondiente al promedio de años de escolaridad (`ID` = 103006)

In [ ]:
data = get_data('103006')

Creación del `DataFrame` con los datos normalizados y visualización de `head` y `tail`

In [ ]:
promedio = pd.DataFrame(data)
promedio

,Country,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Afghanistan,1.490,1.564,1.638,1.712,1.786,1.860,1.920,1.980,2.040,2.100,2.160,2.248,2.336,2.424,2.512,2.600,2.726,2.852,2.978,3.104,3.230,3.310,3.390,3.470,3.550,3.630,3.630,3.780,3.930,3.930
1,Angola,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,4.420,4.420,4.420,4.420,4.420,4.420,4.420,4.481,4.542,4.604,4.665,4.726,4.726,4.788,4.851,4.915,4.980,5.100,5.125,5.174,5.174
2,Albania,7.830,7.828,7.826,7.824,7.822,8.027,8.175,8.323,8.471,8.619,8.767,8.727,8.967,9.020,9.072,9.124,9.177,9.229,9.208,9.287,9.292,9.958,10.025,10.025,10.025,10.025,10.025,10.055,10.055,10.146
3,Andorra,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,9.894,9.894,9.894,10.134,10.515,9.894,10.213,10.251,10.288,10.326,10.364,10.402,10.439,10.477,10.515,10.590,10.590,10.519,10.502,10.502
4,United Arab Emirates,5.623,5.915,6.207,6.500,6.792,7.084,7.323,7.562,7.802,8.041,8.280,8.424,8.569,8.713,8.858,9.119,9.272,9.425,9.578,9.731,9.883,10.036,10.189,10.342,10.495,10.648,10.923,12.111,12.111,12.111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,Samoa,7.602,7.724,7.845,7.966,8.088,8.209,8.331,8.452,8.573,8.695,8.816,8.937,9.059,9.180,9.301,9.423,9.544,9.665,9.787,9.908,10.030,10.490,10.511,10.532,10.553,10.574,10.595,10.595,10.595,10.779
186,Yemen,0.290,0.362,0.434,0.506,0.578,0.650,0.760,0.870,0.980,1.090,1.200,1.332,1.464,1.596,1.728,1.860,2.008,2.156,2.304,2.452,2.600,2.800,3.000,3.000,3.000,3.000,3.000,3.000,3.200,3.200
187,South Africa,6.490,6.600,7.177,7.523,7.869,8.215,8.327,8.438,8.549,8.661,8.772,7.489,8.529,8.768,8.831,8.906,8.957,9.083,9.723,10.137,10.215,9.574,9.821,9.905,9.989,10.134,10.155,10.155,10.241,10.241
188,Zambia,4.680,4.952,5.224,5.496,5.768,6.040,6.008,5.976,5.944,5.912,5.880,5.968,6.056,6.144,6.232,6.320,6.376,6.291,6.488,6.544,6.600,6.674,6.748,6.822,6.769,6.864,6.960,7.056,7.104,7.152


* Dimensiones del `DataFrame`
    * 190 filas
    * 31 columnas

In [ ]:
promedio.shape

(190, 31)

* Nombre de las columnas

In [ ]:
promedio.columns

Index(['Country', '1990', '1991', '1992', '1993', '1994', '1995', '1996',
       '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005',
       '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014',
       '2015', '2016', '2017', '2018', '2019'],
      dtype='object')

#### Calculo del indice de promedio de años de escolaridad por país

Se explica a continuación como se realizo el calculo del indice de promedio de años de escolaridad por pais.
Primeramente se presenta lo que es la formula que se uso:

$$\Huge\Huge\frac{\alpha - \theta}{\gamma - \theta}$$

Donde:

* $\alpha$ : Es el promedio de años de escolaridad en el pais.
* $\theta$ : Es el promedio minimo de años de escolaridad. En el documento __*Human Development Report 2020*__ se especifican 0 años.
* $\gamma$ : Es el promedio maximo de años de escolaridad. En el documento __*Human Development Report 2020*__ se especifican 15 años.


Este calculo se realizo por cada pais en cada uno de los años. 

In [ ]:
 
for year in range(1990, 2020):
    year = str(year)
    indices = []
    for cell in promedio[year]:
        try:
            indices.append((float(cell) - 0) / (15 - 0))
        except:
            indices.append(0)
    promedio[f'Indice_Promedio_Edu_{year}'] = indices
 
promedio = promedio.drop([str(i) for i in range(1990, 2020)], axis=1)

Resultado del calculo del indice del promedio de años de escolaridad por cada país en cada uno de sus periodos.

In [ ]:
promedio

,Country,Indice_Promedio_Edu_1990,Indice_Promedio_Edu_1991,Indice_Promedio_Edu_1992,Indice_Promedio_Edu_1993,Indice_Promedio_Edu_1994,Indice_Promedio_Edu_1995,Indice_Promedio_Edu_1996,Indice_Promedio_Edu_1997,Indice_Promedio_Edu_1998,Indice_Promedio_Edu_1999,Indice_Promedio_Edu_2000,Indice_Promedio_Edu_2001,Indice_Promedio_Edu_2002,Indice_Promedio_Edu_2003,Indice_Promedio_Edu_2004,Indice_Promedio_Edu_2005,Indice_Promedio_Edu_2006,Indice_Promedio_Edu_2007,Indice_Promedio_Edu_2008,Indice_Promedio_Edu_2009,Indice_Promedio_Edu_2010,Indice_Promedio_Edu_2011,Indice_Promedio_Edu_2012,Indice_Promedio_Edu_2013,Indice_Promedio_Edu_2014,Indice_Promedio_Edu_2015,Indice_Promedio_Edu_2016,Indice_Promedio_Edu_2017,Indice_Promedio_Edu_2018,Indice_Promedio_Edu_2019
0,Afghanistan,0.099333,0.104267,0.109200,0.114133,0.119067,0.124000,0.128000,0.132000,0.136000,0.140000,0.144000,0.149867,0.155733,0.161600,0.167467,0.173333,0.181733,0.190133,0.198533,0.206933,0.215333,0.220667,0.226000,0.231333,0.236667,0.242000,0.242000,0.252000,0.262000,0.262000
1,Angola,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.294667,0.294667,0.294667,0.294667,0.294667,0.294667,0.294667,0.298733,0.302800,0.306933,0.311000,0.315067,0.315067,0.319200,0.323400,0.327667,0.332000,0.340000,0.341667,0.344933,0.344933
2,Albania,0.522000,0.521867,0.521733,0.521600,0.521467,0.535133,0.545000,0.554867,0.564733,0.574600,0.584467,0.581800,0.597800,0.601333,0.604800,0.608267,0.611800,0.615267,0.613867,0.619133,0.619467,0.663867,0.668333,0.668333,0.668333,0.668333,0.668333,0.670333,0.670333,0.676400
3,Andorra,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.659600,0.659600,0.659600,0.675600,0.701000,0.659600,0.680867,0.683400,0.685867,0.688400,0.690933,0.693467,0.695933,0.698467,0.701000,0.706000,0.706000,0.701267,0.700133,0.700133
4,United Arab Emirates,0.374867,0.394333,0.413800,0.433333,0.452800,0.472267,0.488200,0.504133,0.520133,0.536067,0.552000,0.561600,0.571267,0.580867,0.590533,0.607933,0.618133,0.628333,0.638533,0.648733,0.658867,0.669067,0.679267,0.689467,0.699667,0.709867,0.728200,0.807400,0.807400,0.807400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,Samoa,0.506800,0.514933,0.523000,0.531067,0.539200,0.547267,0.555400,0.563467,0.571533,0.579667,0.587733,0.595800,0.603933,0.612000,0.620067,0.628200,0.636267,0.644333,0.652467,0.660533,0.668667,0.699333,0.700733,0.702133,0.703533,0.704933,0.706333,0.706333,0.706333,0.718600
186,Yemen,0.019333,0.024133,0.028933,0.033733,0.038533,0.043333,0.050667,0.058000,0.065333,0.072667,0.080000,0.088800,0.097600,0.106400,0.115200,0.124000,0.133867,0.143733,0.153600,0.163467,0.173333,0.186667,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,0.213333,0.213333
187,South Africa,0.432667,0.440000,0.478467,0.501533,0.524600,0.547667,0.555133,0.562533,0.569933,0.577400,0.584800,0.499267,0.568600,0.584533,0.588733,0.593733,0.597133,0.605533,0.648200,0.675800,0.681000,0.638267,0.654733,0.660333,0.665933,0.675600,0.677000,0.677000,0.682733,0.682733
188,Zambia,0.312000,0.330133,0.348267,0.366400,0.384533,0.402667,0.400533,0.398400,0.396267,0.394133,0.392000,0.397867,0.403733,0.409600,0.415467,0.421333,0.425067,0.419400,0.432533,0.436267,0.440000,0.444933,0.449867,0.454800,0.451267,0.457600,0.464000,0.470400,0.473600,0.476800


#### Calculo del Indice de Educacion
Creación de un `DataFrame` vacío para almacenar todos los resultados del calculo 

In [ ]:
header = ['Country']
educacion = pd.DataFrame({}, columns=header)

### Calculo del indice de Educación por país

Se explica a continuación como se realizo el calculo del indice de educación por país.
Primeramente se presenta lo que es la formula que se uso:

$$\Huge\Huge\frac{\alpha + \beta}{2}$$

Donde:

* $\alpha$ : El indice de la expectativa de años de escolaridad en el pais.
* $\beta$ : El indice del promedio de años de escolaridad en el país

Este calculo se realizo por cada pais en cada uno de los años. 


In [ ]:
def calcular_indice(_pais, expect, prom):
    dicci = {
        "Country": _pais
    }
    prom, expect = prom[0][1:], expect[0][1:]
    indice = [(float(prom[i]) + float(expect[i])) / 2 for i in range(0, len(prom))]
 
    for i in range(1990, 2020):
        dicci[str(i)] = indice[i - 1990]
    return dicci
 
 
for country in promedio["Country"]:
    expect = expectativa.loc[expectativa.Country == country].to_numpy()
    prom = promedio.loc[promedio.Country == country].to_numpy()
    if len(expect) > 0 and len(prom) > 0:
        educacion = educacion.append(calcular_indice(country, expect, prom), ignore_index= True)

Resultado del calculo del indice de educación por cada país en cada uno de sus periodos.

In [ ]:
educacion

,Country,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Afghanistan,0.121694,0.133217,0.144767,0.156317,0.167839,0.179389,0.190472,0.201528,0.212611,0.223694,0.234778,0.246767,0.258783,0.270800,0.302011,0.312361,0.324006,0.335622,0.342433,0.351772,0.372444,0.373611,0.390028,0.398222,0.403389,0.405306,0.406056,0.407639,0.412639,0.413667
1,Angola,0.095583,0.090333,0.089833,0.101583,0.104639,0.107694,0.110750,0.113778,0.116833,0.276167,0.288167,0.300139,0.312139,0.324139,0.336111,0.348111,0.362144,0.376150,0.390217,0.404250,0.397700,0.422672,0.435044,0.447422,0.459861,0.472333,0.486639,0.497972,0.499606,0.499606
2,Albania,0.583306,0.587711,0.557089,0.542106,0.541039,0.549956,0.556611,0.569239,0.578728,0.584106,0.586372,0.587678,0.596178,0.603583,0.604761,0.621328,0.627622,0.639967,0.643739,0.650233,0.670844,0.713822,0.739361,0.748778,0.757833,0.752944,0.745417,0.746722,0.743389,0.746422
3,Andorra,0.299972,0.299972,0.299972,0.299972,0.299972,0.299972,0.299972,0.299972,0.299972,0.299972,0.629772,0.629772,0.636383,0.641772,0.650278,0.630522,0.651072,0.652339,0.667156,0.668422,0.669689,0.670956,0.723633,0.714206,0.725361,0.718000,0.722444,0.713022,0.719511,0.719511
4,United Arab Emirates,0.473933,0.493361,0.498761,0.509417,0.528067,0.544272,0.545406,0.545122,0.548289,0.560506,0.572694,0.581744,0.590828,0.599850,0.608933,0.621883,0.631233,0.640556,0.649906,0.659256,0.668544,0.677894,0.687244,0.696567,0.711083,0.734572,0.743072,0.782672,0.802144,0.802144
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,Samoa,0.577150,0.582439,0.587722,0.592978,0.598267,0.603550,0.608839,0.610400,0.619378,0.626667,0.627283,0.632094,0.636967,0.641806,0.654200,0.657683,0.661967,0.666083,0.666039,0.677044,0.691528,0.697111,0.694256,0.697122,0.699961,0.697828,0.698472,0.700944,0.700944,0.712911
185,Yemen,0.218861,0.221650,0.224439,0.227228,0.230017,0.232833,0.236889,0.240944,0.243611,0.249056,0.258111,0.267900,0.277689,0.287478,0.296989,0.302583,0.305378,0.308172,0.310633,0.318678,0.325250,0.342694,0.334917,0.344694,0.342694,0.341694,0.340667,0.340667,0.347333,0.350250
186,South Africa,0.532194,0.551833,0.581317,0.603100,0.616522,0.636000,0.639289,0.642572,0.645856,0.649144,0.652428,0.609244,0.643494,0.651017,0.652700,0.654783,0.656039,0.659822,0.680739,0.694122,0.696278,0.674494,0.684922,0.695361,0.705356,0.720161,0.718167,0.718167,0.721033,0.724450
187,Zambia,0.364833,0.380178,0.395550,0.410894,0.426267,0.441611,0.446822,0.452061,0.457272,0.462511,0.467722,0.476961,0.486172,0.495411,0.504622,0.513861,0.522006,0.513144,0.521933,0.523967,0.526028,0.526133,0.528572,0.531844,0.532467,0.538050,0.543639,0.549228,0.553244,0.557233


### Calculo del Indice de Salud
Obtencion de los registros del indicador correspondiente a la expectativa de años de vida al nacer (`ID` = 69206)

In [ ]:
data = get_data('69206')

Creación del `DataFrame` con los datos normalizados y visualización de `head` y `tail`

In [ ]:
salud = pd.DataFrame(data)
salud

,Country,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Afghanistan,50.331,50.999,51.641,52.256,52.842,53.398,53.924,54.424,54.906,55.376,55.841,56.308,56.784,57.271,57.772,58.290,58.826,59.375,59.930,60.484,61.028,61.553,62.054,62.525,62.966,63.377,63.763,64.130,64.486,64.83
1,Angola,45.306,45.271,45.230,45.201,45.201,45.246,45.350,45.519,45.763,46.093,46.522,47.059,47.702,48.440,49.263,50.165,51.143,52.177,53.243,54.311,55.350,56.330,57.236,58.054,58.776,59.398,59.925,60.379,60.782,61.15
2,Albania,71.836,71.803,71.802,71.860,71.992,72.205,72.495,72.838,73.208,73.587,73.955,74.288,74.579,74.828,75.039,75.228,75.423,75.646,75.912,76.221,76.562,76.914,77.252,77.554,77.813,78.025,78.194,78.333,78.458,78.57
3,Andorra,76.517,76.682,76.854,77.030,77.213,77.414,77.644,77.909,78.207,78.531,78.867,79.198,79.505,79.776,80.005,80.192,80.343,80.471,80.589,80.703,80.818,80.935,81.054,81.173,81.294,81.416,81.540,81.663,81.786,81.91
4,United Arab Emirates,71.939,72.208,72.466,72.715,72.957,73.194,73.428,73.657,73.883,74.106,74.327,74.544,74.758,74.968,75.174,75.376,75.573,75.767,75.957,76.145,76.332,76.521,76.711,76.903,77.095,77.285,77.470,77.647,77.814,77.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186,Samoa,66.281,66.470,66.655,66.842,67.037,67.249,67.491,67.764,68.067,68.399,68.748,69.103,69.451,69.781,70.089,70.374,70.642,70.901,71.158,71.412,71.663,71.906,72.136,72.351,72.549,72.730,72.895,73.046,73.187,73.32
187,Yemen,57.346,57.730,58.047,58.318,58.566,58.817,59.096,59.415,59.782,60.204,60.683,61.216,61.781,62.358,62.931,63.481,63.997,64.470,64.892,65.255,65.549,65.768,65.920,66.016,66.066,66.085,66.087,66.086,66.096,66.13
188,South Africa,63.307,63.384,63.247,62.894,62.331,61.561,60.595,59.489,58.315,57.144,56.048,55.089,54.310,53.749,53.444,53.447,53.795,54.452,55.360,56.460,57.669,58.895,60.060,61.099,61.968,62.649,63.153,63.538,63.857,64.13
189,Zambia,49.249,48.125,46.987,45.919,44.983,44.242,43.735,43.461,43.413,43.594,44.000,44.615,45.400,46.322,47.354,48.496,49.757,51.134,52.606,54.130,55.655,57.126,58.502,59.746,60.831,61.737,62.464,63.043,63.510,63.89


* Dimensiones del `DataFrame`
    * 191 filas
    * 31 columnas

In [ ]:
salud.shape

(191, 31)

* Nombre de las columnas

In [ ]:
salud.columns

Index(['Country', '1990', '1991', '1992', '1993', '1994', '1995', '1996',
       '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005',
       '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014',
       '2015', '2016', '2017', '2018', '2019'],
      dtype='object')

#### Calculo del Indice de Salud por paises.

Se explica a continuación como se realizo el calculo del indice de salud por pais.
Primeramente se presenta lo que es la formula que se uso:

$$\Huge\Huge\frac{\alpha - \theta}{\gamma - \theta}$$

Donde:

* $\alpha$ : Es la esperanza de vida en el pais.
* $\theta$ : Es la esperanza minima de vida. En el documento __*Human Development Report 2020*__ se especifican 20 años.
* $\gamma$ : Es la esperanza maxima de vida. En el documento __*Human Development Report 2020*__ se especifican 85 años.


Este calculo se realizo por cada pais en cada uno de los años.

In [ ]:
#Calculo del "Indice_Salud"
for i in range(1990,2020):
    ind = str(i)
    salud[ind] = (salud[ind]-20)/(85-20)

Resultado del calculo del indice de salud por cada país en cada uno de sus periodos.

In [ ]:
salud

,Country,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Afghanistan,0.466631,0.476908,0.486785,0.496246,0.505262,0.513815,0.521908,0.529600,0.537015,0.544246,0.551400,0.558585,0.565908,0.573400,0.581108,0.589077,0.597323,0.605769,0.614308,0.622831,0.631200,0.639277,0.646985,0.654231,0.661015,0.667338,0.673277,0.678923,0.684400,0.689692
1,Angola,0.389323,0.388785,0.388154,0.387708,0.387708,0.388400,0.390000,0.392600,0.396354,0.401431,0.408031,0.416292,0.426185,0.437538,0.450200,0.464077,0.479123,0.495031,0.511431,0.527862,0.543846,0.558923,0.572862,0.585446,0.596554,0.606123,0.614231,0.621215,0.627415,0.633077
2,Albania,0.797477,0.796969,0.796954,0.797846,0.799877,0.803154,0.807615,0.812892,0.818585,0.824415,0.830077,0.835200,0.839677,0.843508,0.846754,0.849662,0.852662,0.856092,0.860185,0.864938,0.870185,0.875600,0.880800,0.885446,0.889431,0.892692,0.895292,0.897431,0.899354,0.901077
3,Andorra,0.869492,0.872031,0.874677,0.877385,0.880200,0.883292,0.886831,0.890908,0.895492,0.900477,0.905646,0.910738,0.915462,0.919631,0.923154,0.926031,0.928354,0.930323,0.932138,0.933892,0.935662,0.937462,0.939292,0.941123,0.942985,0.944862,0.946769,0.948662,0.950554,0.952462
4,United Arab Emirates,0.799062,0.803200,0.807169,0.811000,0.814723,0.818369,0.821969,0.825492,0.828969,0.832400,0.835800,0.839138,0.842431,0.845662,0.848831,0.851938,0.854969,0.857954,0.860877,0.863769,0.866646,0.869554,0.872477,0.875431,0.878385,0.881308,0.884154,0.886877,0.889446,0.891846
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186,Samoa,0.712015,0.714923,0.717769,0.720646,0.723646,0.726908,0.730631,0.734831,0.739492,0.744600,0.749969,0.755431,0.760785,0.765862,0.770600,0.774985,0.779108,0.783092,0.787046,0.790954,0.794815,0.798554,0.802092,0.805400,0.808446,0.811231,0.813769,0.816092,0.818262,0.820308
187,Yemen,0.574554,0.580462,0.585338,0.589508,0.593323,0.597185,0.601477,0.606385,0.612031,0.618523,0.625892,0.634092,0.642785,0.651662,0.660477,0.668938,0.676877,0.684154,0.690646,0.696231,0.700754,0.704123,0.706462,0.707938,0.708708,0.709000,0.709031,0.709015,0.709169,0.709692
188,South Africa,0.666262,0.667446,0.665338,0.659908,0.651246,0.639400,0.624538,0.607523,0.589462,0.571446,0.554585,0.539831,0.527846,0.519215,0.514523,0.514569,0.519923,0.530031,0.544000,0.560923,0.579523,0.598385,0.616308,0.632292,0.645662,0.656138,0.663892,0.669815,0.674723,0.678923
189,Zambia,0.449985,0.432692,0.415185,0.398754,0.384354,0.372954,0.365154,0.360938,0.360200,0.362985,0.369231,0.378692,0.390769,0.404954,0.420831,0.438400,0.457800,0.478985,0.501631,0.525077,0.548538,0.571169,0.592338,0.611477,0.628169,0.642108,0.653292,0.662200,0.669385,0.675231


### Calculo del Indice de Salud
Obtencion de los registros del indicador correspondiente a la expectativa de años de vida al nacer (`ID` = 69206)

In [ ]:
data = get_data('195706')

Creación del `DataFrame` con los datos normalizados y visualización de `head` y `tail`

In [ ]:
ingreso = pd.DataFrame(data)
ingreso

,Country,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Afghanistan,2477.906,2059.950,1921.695,1320.535,950.897,1344.364,1223.441,1130.017,1057.778,985.222,904.047,820.163,1290.944,1334.268,1311.793,1389.895,1478.934,1829.792,1672.101,1900.219,1917.395,2013.614,2164.641,2229.907,2214.414,2128.162,2134.866,2229.658,2217.176,2229.362
1,Angola,4823.397,5380.463,2064.354,2024.850,1550.029,3396.198,3357.465,3962.468,3884.616,3507.965,3673.613,3882.750,4638.499,4662.816,4907.245,5365.880,5646.012,6329.901,6460.903,6966.947,6913.161,6887.004,7282.050,7478.856,7704.368,7652.152,7189.032,6861.581,6360.551,6104.055
2,Albania,4937.523,3496.390,3207.627,3684.634,4102.593,4771.803,5252.751,4681.337,5115.353,5715.555,6139.337,6660.765,6923.306,7337.603,7750.245,8198.824,8830.432,9433.901,10019.350,10240.670,10774.722,11237.447,11365.140,11806.358,11951.263,12273.473,12753.307,13071.095,13636.864,13998.300
3,Andorra,45393.316,44773.183,43487.951,41568.988,41401.230,41761.304,43356.786,47312.497,48964.850,50791.416,50582.018,51349.775,52559.056,56430.901,58307.002,60537.302,61622.550,60386.327,54423.801,52042.239,49261.522,47366.247,47347.416,48486.415,50567.870,51779.832,53245.151,54371.345,55253.539,56000.303
4,United Arab Emirates,102433.229,96250.378,93043.562,92505.415,96784.515,101303.368,101490.336,103567.129,99915.860,98569.282,104640.376,103519.148,96736.703,97761.511,98236.247,92443.496,88502.479,79459.366,69814.622,59017.231,54911.287,56152.975,57447.351,60007.281,62499.798,65528.563,66881.303,67667.530,67195.144,67462.095
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186,Samoa,5938.879,5697.935,5433.785,5682.170,3700.967,3931.255,4370.056,4379.692,4477.627,4589.567,4880.928,5271.771,5009.707,5243.890,5467.568,5662.869,5653.741,5992.421,5955.895,5915.953,6062.656,5969.899,5703.125,5693.886,5780.675,6237.246,6409.360,6287.699,6126.974,6308.652
187,Yemen,2958.129,2788.322,2762.329,2749.622,2681.535,2915.890,3232.027,3325.112,3504.726,3426.164,3611.962,3754.576,3809.142,3874.708,3892.670,3979.725,4179.637,4209.567,4213.487,4363.312,4304.374,3493.180,3600.009,3607.262,3136.621,2194.613,1845.552,1619.640,1563.988,1593.704
188,South Africa,9974.994,9725.792,9312.598,9212.297,9312.299,9387.381,9582.037,9660.656,9537.183,9617.403,9881.364,9941.190,10238.534,10388.185,10807.167,11233.344,11711.353,12010.290,12251.733,11999.965,12195.419,12346.453,12403.797,12523.410,12549.480,12528.094,12356.603,12322.437,12231.840,12129.230
189,Zambia,2015.630,1866.013,1818.721,1940.969,1755.783,1763.636,1836.819,1853.643,1787.271,1848.575,1879.253,1934.226,1978.912,2077.097,2088.712,2154.750,2225.791,2300.915,2499.270,2800.141,2872.212,2873.238,3333.817,3389.723,3263.274,3403.717,3237.725,3330.785,3365.698,3325.519


* Dimensiones del `DataFrame`
    * 191 filas
    * 31 columnas

In [ ]:
ingreso.shape

(191, 31)

* Nombre de las columnas

In [ ]:
ingreso.columns

Index(['Country', '1990', '1991', '1992', '1993', '1994', '1995', '1996',
       '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005',
       '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014',
       '2015', '2016', '2017', '2018', '2019'],
      dtype='object')

#### Calculo del Indice de Ingreso por paises.

Se explica a continuación como se realizo el calculo del indice de ingreso por pais.
Primeramente se presenta lo que es la formula que se uso:

$$\Huge\Huge\frac{\log(\alpha) - \log(\theta)}{\log(\gamma) - \log(\theta)}$$

Donde:

* $\alpha$ : Es el ingreso nacional bruto.
* $\theta$ : Es el ingreso nacional bruto minimo. En el documento __*Human Development Report 2020*__ se especifica 75,000.
* $\gamma$ : Es el ingreso nacional bruto maximo. En el documento __*Human Development Report 2020*__ se especifica 100,000.


Este calculo se realizo por cada pais en cada uno de los años.

In [ ]:
#Calculo del "Indice_Ingreso"
for i in range(1990,2020):
    ind = str(i)
    val = ingreso[ind]
    ingreso[ind] = [(np.log(i)-np.log(100))/(np.log(75000)-np.log(100)) 
    if i != 0 else 0 for i in val]

Resultado del calculo del indice de ingreso por cada país en cada uno de sus periodos.

In [ ]:
ingreso

,Country,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Afghanistan,0.484889,0.456984,0.446489,0.389818,0.340213,0.392519,0.378282,0.366283,0.356304,0.345570,0.332581,0.317872,0.386394,0.391381,0.388814,0.397550,0.406930,0.439087,0.425474,0.444792,0.446151,0.453547,0.464472,0.468959,0.467906,0.461905,0.462380,0.468942,0.468094,0.468922
1,Angola,0.585502,0.602011,0.457306,0.454388,0.414022,0.532508,0.530775,0.555802,0.552805,0.537399,0.544369,0.552732,0.579597,0.580387,0.588105,0.601601,0.609289,0.626559,0.629654,0.641045,0.639874,0.639301,0.647727,0.651755,0.656242,0.655215,0.645785,0.638743,0.627289,0.621071
2,Albania,0.589034,0.536900,0.523879,0.544821,0.561052,0.583877,0.598383,0.580986,0.594379,0.611138,0.621942,0.634256,0.640095,0.648875,0.657139,0.665639,0.676849,0.686835,0.695929,0.699230,0.706909,0.713261,0.714967,0.720721,0.722563,0.726582,0.732375,0.736093,0.742494,0.746445
3,Andorra,0.924151,0.922074,0.917674,0.910857,0.910246,0.911554,0.917218,0.930407,0.935592,0.941124,0.940500,0.942776,0.946292,0.957029,0.961969,0.967640,0.970324,0.967262,0.951558,0.944799,0.936505,0.930578,0.930518,0.934109,0.940458,0.944036,0.948251,0.951413,0.953844,0.955872
4,United Arab Emirates,1.047088,1.037683,1.032565,1.031688,1.038519,1.045412,1.045691,1.048750,1.043329,1.041279,1.050308,1.048681,1.038444,1.040036,1.040768,1.031587,1.025006,1.008725,0.989178,0.963798,0.952905,0.956283,0.959726,0.966311,0.972459,0.979607,0.982694,0.984459,0.983401,0.984000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186,Samoa,0.616928,0.610671,0.603501,0.610253,0.545489,0.554608,0.570592,0.570925,0.574265,0.577995,0.587293,0.598929,0.591226,0.598128,0.604437,0.609739,0.609495,0.618283,0.617360,0.616343,0.620043,0.617714,0.610809,0.610564,0.612849,0.624332,0.628444,0.625549,0.621638,0.626052
187,Yemen,0.511647,0.502717,0.501303,0.500606,0.496818,0.509475,0.525024,0.529313,0.537260,0.533835,0.541812,0.547662,0.549841,0.552419,0.553118,0.556459,0.563862,0.564940,0.565081,0.570359,0.568304,0.536761,0.541311,0.541615,0.520497,0.466549,0.440382,0.420658,0.415377,0.418220
188,South Africa,0.695259,0.691437,0.684880,0.683244,0.684875,0.686088,0.689188,0.690422,0.688479,0.689745,0.693835,0.694746,0.699198,0.701390,0.707363,0.713205,0.719500,0.723308,0.726314,0.723178,0.725618,0.727477,0.728177,0.729627,0.729941,0.729684,0.727602,0.727183,0.726069,0.724796
189,Zambia,0.453698,0.442048,0.438170,0.447997,0.432850,0.433524,0.439666,0.441043,0.435535,0.440630,0.443116,0.447471,0.450921,0.458236,0.459078,0.463780,0.468680,0.473694,0.486186,0.503356,0.507195,0.507249,0.529708,0.532220,0.526477,0.532842,0.525290,0.529570,0.531145,0.529331


### Calculo del Indice de Desarrollo Humano (`IDH`)
Apoyandose de los datos obtenidos y calculados con anterioridad se procederá a calcular el `IDH`, donde, de manera similar al Indice de Educación, se generarán un dataframe vacio que contendra el Indice calculado 

In [ ]:
header = ['Country']
indice_desarrollo_humano = pd.DataFrame({}, columns=header)

#### Calculo del Indice de Desarrollo Humano por paises.

Se explica a continuación como se realizo el calculo del indice de salud por pais.
Primeramente se presenta lo que es la formula que se uso:

$$\Huge\Huge{IDH=\left(I_{Salud}+I_{Educacion}+I_{Ingreso}\right)^{\frac{1}{3}}}$$

Donde:

* $IDH$ : Es el indice de desarrollo humano
* $I_{Salud}$ : Es el indice de salud.
* $I_{Educacion}$ : Es el indice de educación
* $I_{Ingreso}$ : Es el indice de ingreso.


Este calculo se realizo por cada pais en cada uno de los años.

In [ ]:
def IDH(_pais, salud, educacion, ingreso):
    dicci = {
        "Country": _pais
    }
    s, e, i = salud[0][1:], educacion[0][1:], ingreso[0][1:]
    idh = np.power((s * e * i), (1/3))

    for i in range(1990, 2020):
        dicci[str(i)] = idh[i - 1990]
    return dicci


for country in educacion["Country"]:
    i_salud = salud.loc[salud.Country == country].to_numpy()
    i_educacion = educacion.loc[educacion.Country == country].to_numpy()
    i_ingreso = ingreso.loc[ingreso.Country == country].to_numpy()
    if len(i_salud) > 0 and len(i_educacion) > 0 and len(i_ingreso) > 0:
        indice_desarrollo_humano = indice_desarrollo_humano.append(
            IDH(country, i_salud, i_educacion, i_ingreso),
            ignore_index= True)

Resultado del calculo del indice de desarrollo humano por cada país en cada uno de sus periodos.

In [ ]:
indice_desarrollo_humano

,Country,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Afghanistan,0.301969,0.307349,0.315698,0.311545,0.306704,0.330741,0.335027,0.339391,0.343925,0.347799,0.350488,0.352541,0.383917,0.393159,0.408640,0.418222,0.428641,0.446927,0.447313,0.460182,0.471596,0.476699,0.489384,0.496208,0.499687,0.499912,0.501872,0.506297,0.509414,0.511449
1,Angola,0.279302,0.276516,0.251700,0.261568,0.256099,0.281362,0.284079,0.291727,0.294718,0.390565,0.400015,0.410278,0.425622,0.434999,0.446458,0.459768,0.472843,0.488636,0.500877,0.515252,0.517260,0.532542,0.544492,0.554749,0.564651,0.572441,0.577930,0.582449,0.581500,0.581311
2,Albania,0.649509,0.631199,0.614983,0.617664,0.623857,0.636526,0.645523,0.645403,0.655441,0.665159,0.671450,0.677743,0.684296,0.691293,0.695557,0.705670,0.712835,0.721955,0.727705,0.732641,0.744501,0.763920,0.775068,0.781797,0.786781,0.787499,0.787709,0.790128,0.791796,0.794782
3,Andorra,0.622343,0.622481,0.622117,0.621213,0.621738,0.622763,0.624882,0.628824,0.631068,0.633481,0.812519,0.814695,0.819964,0.826621,0.832744,0.826697,0.837053,0.837305,0.839552,0.838615,0.837209,0.836501,0.858384,0.856293,0.863242,0.861973,0.865613,0.863365,0.867289,0.868483
4,United Arab Emirates,0.734672,0.743620,0.746320,0.752571,0.764489,0.775088,0.776830,0.778562,0.779812,0.786128,0.795147,0.799963,0.802527,0.808033,0.813295,0.817600,0.820898,0.821481,0.821023,0.818740,0.820365,0.826067,0.831772,0.838366,0.846887,0.859155,0.864289,0.880806,0.888589,0.889568
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,Samoa,0.632903,0.633538,0.633789,0.638886,0.618115,0.624299,0.633153,0.635029,0.640719,0.646092,0.651309,0.658846,0.659241,0.664940,0.672920,0.677357,0.679936,0.685767,0.686561,0.691083,0.698499,0.700591,0.698044,0.699870,0.702575,0.707027,0.709533,0.709951,0.709095,0.715387
185,Yemen,0.400704,0.401410,0.403833,0.406271,0.407771,0.413770,0.421354,0.426049,0.431074,0.434864,0.444002,0.453119,0.461268,0.469496,0.476949,0.482934,0.488472,0.492021,0.494923,0.502055,0.505966,0.505955,0.504070,0.509378,0.501877,0.483497,0.473814,0.466629,0.467713,0.470202
186,South Africa,0.627028,0.633859,0.642229,0.647862,0.650282,0.653436,0.650426,0.645951,0.639971,0.634846,0.630839,0.611353,0.619276,0.618918,0.619327,0.621706,0.626085,0.632437,0.645508,0.655433,0.664029,0.664650,0.674878,0.684557,0.692735,0.701218,0.702650,0.704598,0.706892,0.709058
187,Zambia,0.420746,0.417393,0.415938,0.418700,0.413920,0.414861,0.415506,0.415946,0.415509,0.419786,0.424555,0.432359,0.440829,0.451322,0.460243,0.470988,0.482032,0.488300,0.503038,0.517369,0.526983,0.534188,0.549419,0.557295,0.560509,0.568866,0.571398,0.577504,0.581570,0.583990


### Traduccion de los nombres de cada país de ingles a español
Por cuestiones de comodidad, todos los nombres presentes en la columna `Country`, para ello se definio una nueva función que hará uso del modulo `google_trans_new`

In [ ]:
 %%capture
!pip install google_trans_new
from google_trans_new import google_translator
 
def translate(df: pd.DataFrame) -> pd.DataFrame:
    translator = google_translator()
    regs = df.Country.count()
    for i in range(0, regs):
        pais = translator.translate(df.Country[i], lang_tgt='es', lang_src='en')
        df.Country[i] = pais.strip() if type(pais) is not list else pais[1].strip()
    return df

Posteriormente se llamara a la función anterior y se enciara como parametro el `DataFrame` llamado `indice_desarrollo_humano`

In [ ]:
%%capture
indice_desarrollo_humano = translate(indice_desarrollo_humano)

### Conservación de los indices calculados en un archivo CSV 
Una vez concretados todos los procedimientos planeados se conservarán los indices en un nuevo archivo `.csv`

In [ ]:
indice_desarrollo_humano.head()

,Country,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Afganistán,0.301969,0.307349,0.315698,0.311545,0.306704,0.330741,0.335027,0.339391,0.343925,0.347799,0.350488,0.352541,0.383917,0.393159,0.408640,0.418222,0.428641,0.446927,0.447313,0.460182,0.471596,0.476699,0.489384,0.496208,0.499687,0.499912,0.501872,0.506297,0.509414,0.511449
1,Angola,0.279302,0.276516,0.251700,0.261568,0.256099,0.281362,0.284079,0.291727,0.294718,0.390565,0.400015,0.410278,0.425622,0.434999,0.446458,0.459768,0.472843,0.488636,0.500877,0.515252,0.517260,0.532542,0.544492,0.554749,0.564651,0.572441,0.577930,0.582449,0.581500,0.581311
2,Albania,0.649509,0.631199,0.614983,0.617664,0.623857,0.636526,0.645523,0.645403,0.655441,0.665159,0.671450,0.677743,0.684296,0.691293,0.695557,0.705670,0.712835,0.721955,0.727705,0.732641,0.744501,0.763920,0.775068,0.781797,0.786781,0.787499,0.787709,0.790128,0.791796,0.794782
3,Andorra,0.622343,0.622481,0.622117,0.621213,0.621738,0.622763,0.624882,0.628824,0.631068,0.633481,0.812519,0.814695,0.819964,0.826621,0.832744,0.826697,0.837053,0.837305,0.839552,0.838615,0.837209,0.836501,0.858384,0.856293,0.863242,0.861973,0.865613,0.863365,0.867289,0.868483
4,Emiratos Árabes Unidos,0.734672,0.743620,0.746320,0.752571,0.764489,0.775088,0.776830,0.778562,0.779812,0.786128,0.795147,0.799963,0.802527,0.808033,0.813295,0.817600,0.820898,0.821481,0.821023,0.818740,0.820365,0.826067,0.831772,0.838366,0.846887,0.859155,0.864289,0.880806,0.888589,0.889568


### Inspección del DataSet antes de proceder a guardarlo.

Antes de guardar los resultados se eliminaran los datos `NUll` del DataSet para futuros casos. En el calculo que se realizo en el Notebook se transformaron los `Null` por valores _0_. Asi que se procedio a remplazarlos y eliminarlos.

In [ ]:
IDH = indice_desarrollo_humano.replace([0],np.nan).copy()

In [ ]:
#Se observa la cantidad de registros que cuentan con Null
print(f'El total de registros con nulos son: {IDH.shape[0] - IDH.dropna().shape[0]}')

El total de registros con nulos son: 13


Se muestran los registros con algun valor `Null`. Los 13 registros con algun `Null` son los siguientes

In [ ]:
indices_sin_nulos = IDH.dropna().index.to_list()
indices_con_nulos = IDH.index.to_list()
indices_borrados = []

for i in indices_con_nulos:
    if not i in indices_sin_nulos:
        indices_borrados.append(i)

IDH.loc[indices_borrados]

,Country,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
7,Antigua y Barbuda,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.531063,0.574051,0.575020,0.578806,0.582161,0.763859,0.770538,0.775523,0.774367,0.767284,0.762708,0.755184,0.758845,0.759764,0.759960,0.762378,0.764867,0.767829,0.771866,0.777744
19,Bosnia y Herzegovina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.678859,0.683652,0.689374,0.694567,0.700858,0.706458,0.712368,0.719137,0.726119,0.726644,0.720565,0.728776,0.747527,0.754910,0.758147,0.761423,0.771734,0.773596,0.777271,0.779776
39,Cabo Verde,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.505514,0.568622,0.573047,0.582725,0.587513,0.590432,0.596315,0.611058,0.617259,0.625499,0.629600,0.631946,0.638646,0.643800,0.647199,0.653861,0.655804,0.657180,0.659770,0.663338,0.665255
52,Eritrea,NaN,NaN,NaN,NaN,NaN,0.295244,0.298275,0.301001,0.307599,0.319691,0.331720,0.329067,0.340322,0.341346,0.357810,0.427460,0.428205,0.430111,0.426377,0.435497,0.436277,0.436223,0.441496,0.445729,0.457469,0.454289,0.456298,0.453757,0.456430,0.458570
59,Micronesia (estados federados de),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.546267,0.552693,0.558644,0.565249,0.570256,0.577485,0.583299,0.588618,0.592591,0.596411,0.600627,0.603869,0.604708,0.604735,0.604417,0.612161,0.614078,0.616288,0.617697,0.619878
67,Guinea Ecuatorial,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.524721,0.534190,0.545701,0.547030,0.554004,0.565470,0.575636,0.581566,0.582819,0.585146,0.576477,0.579509,0.584007,0.584936,0.586266,0.589398,0.587979,0.584050,0.581614,0.592306
112,Islas Marshall,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.528429,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.546313,0.544685,0.548786,0.551464,0.556703,0.557915,0.699394,0.701574,0.703723
117,Montenegro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.773358,0.783567,0.794416,0.796542,0.801813,0.806021,0.806973,0.810596,0.812509,0.815888,0.817631,0.821947,0.826463,0.828913
142,"Palestina, estado de",NaN,NaN,NaN,NaN,NaN,0.495006,0.500288,0.511973,0.524294,0.532801,0.526078,0.525838,0.521607,0.532817,0.650763,0.658511,0.659261,0.669438,0.673707,0.678738,0.683585,0.691745,0.697860,0.695260,0.696745,0.700558,0.703673,0.706298,0.707726,0.707681
155,Sudán del Sur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.332067,0.333075,0.410309,0.430155,0.396619,0.427914,0.427644,0.424856,0.420524,0.426461,0.428717,0.432623


Ahora se genera un DataSet sin `Null`

In [ ]:
IDH_sin_nan = IDH.copy().dropna().reset_index(drop=True)

Ahora se volvera a ordenar el DataSet para que aparescan segun el Top del año __2019__.


Para realizar esta activad fue necesario de realizar las siguientes operaciones

In [ ]:
IDH_final = IDH_sin_nan.sort_values("2019",ascending=False).copy()

IDH_final.index = list(range(1,177))

IDH_final['Top'] = IDH_final.index

IDH_final.head()

,Country,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,Top
1,Australia,0.870693,0.871223,0.872507,0.876370,0.879274,0.895285,0.902863,0.912976,0.915095,0.921715,0.925590,0.924032,0.930376,0.933444,0.932518,0.927578,0.918483,0.922666,0.936733,0.941303,0.946261,0.951179,0.958174,0.953330,0.955447,0.985071,0.985249,0.984336,0.978750,0.980079,1
2,Irlanda,0.773121,0.777016,0.782391,0.791301,0.798486,0.805037,0.810631,0.818775,0.850570,0.858953,0.867340,0.873339,0.880938,0.888775,0.897943,0.903705,0.905651,0.909234,0.910882,0.906249,0.901003,0.902290,0.908060,0.917159,0.930020,0.941829,0.950749,0.954173,0.958234,0.961968,2
3,Suecia,0.821292,0.823135,0.826142,0.845684,0.854699,0.862009,0.869172,0.879829,0.898823,0.905771,0.910296,0.913228,0.913845,0.918389,0.901075,0.903963,0.907757,0.909915,0.906689,0.905253,0.911419,0.912481,0.913630,0.934052,0.937303,0.943468,0.947553,0.949389,0.950488,0.959108,3
4,Islandia,0.806552,0.819830,0.822983,0.823512,0.828595,0.831411,0.836025,0.845594,0.855499,0.859629,0.866726,0.872147,0.880925,0.889973,0.890573,0.897988,0.901959,0.907381,0.902520,0.900715,0.903505,0.912174,0.922829,0.940489,0.945808,0.946763,0.953652,0.953697,0.955974,0.959040,4
5,Noruega,0.848578,0.854332,0.860797,0.868812,0.883097,0.881293,0.886856,0.892971,0.904391,0.909133,0.915176,0.914329,0.916796,0.922500,0.932123,0.930795,0.934347,0.936453,0.936565,0.936863,0.940267,0.941942,0.940902,0.944327,0.943832,0.946818,0.949745,0.954415,0.956131,0.957642,5


Ahora procedemos a realizar el ultimo analisis exploratorio

In [ ]:
# Cabecera del DataFrame
IDH_final.head()

,Country,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,Top
1,Australia,0.870693,0.871223,0.872507,0.876370,0.879274,0.895285,0.902863,0.912976,0.915095,0.921715,0.925590,0.924032,0.930376,0.933444,0.932518,0.927578,0.918483,0.922666,0.936733,0.941303,0.946261,0.951179,0.958174,0.953330,0.955447,0.985071,0.985249,0.984336,0.978750,0.980079,1
2,Irlanda,0.773121,0.777016,0.782391,0.791301,0.798486,0.805037,0.810631,0.818775,0.850570,0.858953,0.867340,0.873339,0.880938,0.888775,0.897943,0.903705,0.905651,0.909234,0.910882,0.906249,0.901003,0.902290,0.908060,0.917159,0.930020,0.941829,0.950749,0.954173,0.958234,0.961968,2
3,Suecia,0.821292,0.823135,0.826142,0.845684,0.854699,0.862009,0.869172,0.879829,0.898823,0.905771,0.910296,0.913228,0.913845,0.918389,0.901075,0.903963,0.907757,0.909915,0.906689,0.905253,0.911419,0.912481,0.913630,0.934052,0.937303,0.943468,0.947553,0.949389,0.950488,0.959108,3
4,Islandia,0.806552,0.819830,0.822983,0.823512,0.828595,0.831411,0.836025,0.845594,0.855499,0.859629,0.866726,0.872147,0.880925,0.889973,0.890573,0.897988,0.901959,0.907381,0.902520,0.900715,0.903505,0.912174,0.922829,0.940489,0.945808,0.946763,0.953652,0.953697,0.955974,0.959040,4
5,Noruega,0.848578,0.854332,0.860797,0.868812,0.883097,0.881293,0.886856,0.892971,0.904391,0.909133,0.915176,0.914329,0.916796,0.922500,0.932123,0.930795,0.934347,0.936453,0.936565,0.936863,0.940267,0.941942,0.940902,0.944327,0.943832,0.946818,0.949745,0.954415,0.956131,0.957642,5


In [ ]:
# Cola del DataFrame
IDH_final.tail()

,Country,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,Top
172,Mali,0.234375,0.241457,0.246009,0.252800,0.259723,0.265495,0.274233,0.284566,0.294386,0.305908,0.312247,0.324789,0.334129,0.347347,0.356492,0.367352,0.376420,0.370945,0.394383,0.401402,0.408147,0.413097,0.413020,0.412537,0.419227,0.417214,0.422185,0.426560,0.431067,0.433518,172
173,Burundi,0.299301,0.302422,0.298700,0.299125,0.296350,0.292451,0.288604,0.289941,0.293805,0.295075,0.299774,0.300530,0.311031,0.319079,0.329641,0.337295,0.359477,0.371186,0.383916,0.400544,0.410902,0.419464,0.425516,0.432090,0.438292,0.437007,0.437587,0.434400,0.431241,0.433409,173
174,Chad,0.233428,0.237697,0.241455,0.235156,0.240500,0.241891,0.243316,0.251244,0.263297,0.265308,0.293492,0.301219,0.306730,0.305180,0.324177,0.327223,0.329798,0.340243,0.345601,0.361140,0.369195,0.379957,0.388415,0.394184,0.401449,0.397951,0.392793,0.395799,0.397360,0.397799,174
175,República Centroafricana,0.333983,0.328430,0.315337,0.315314,0.316966,0.319069,0.315468,0.318079,0.319967,0.323357,0.324812,0.326722,0.328359,0.327780,0.331477,0.336127,0.341691,0.347123,0.352189,0.356316,0.365116,0.373587,0.380703,0.362842,0.367845,0.374875,0.382174,0.390715,0.394578,0.397197,175
176,Níger,0.219993,0.223406,0.226015,0.230536,0.234931,0.239599,0.243946,0.248188,0.255459,0.259048,0.262070,0.268077,0.273079,0.276286,0.284835,0.293682,0.300344,0.305553,0.314428,0.320607,0.331136,0.338438,0.349674,0.357145,0.365277,0.371820,0.378127,0.386300,0.390611,0.393708,176


In [ ]:
# Informacion de DataFrame
IDH_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 176 entries, 1 to 176
Data columns (total 32 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Country  176 non-null    object 
 1   1990     176 non-null    float64
 2   1991     176 non-null    float64
 3   1992     176 non-null    float64
 4   1993     176 non-null    float64
 5   1994     176 non-null    float64
 6   1995     176 non-null    float64
 7   1996     176 non-null    float64
 8   1997     176 non-null    float64
 9   1998     176 non-null    float64
 10  1999     176 non-null    float64
 11  2000     176 non-null    float64
 12  2001     176 non-null    float64
 13  2002     176 non-null    float64
 14  2003     176 non-null    float64
 15  2004     176 non-null    float64
 16  2005     176 non-null    float64
 17  2006     176 non-null    float64
 18  2007     176 non-null    float64
 19  2008     176 non-null    float64
 20  2009     176 non-null    float64
 21  2010     176 non

In [ ]:
# Dimensiones del DataFrame
IDH_final.shape

(176, 32)

Teniendo _176 filas_ y _32 columnas_. 

## Consultas al DataFrame.

Por ultimo en el analisis se apoyo de una herramienta denominada __widgets__ para poder hacer una analisis exploratorio un poco mas interactivo. 


In [ ]:
# Importaciones necesarias

import ipywidgets as widgets
from ipywidgets import interact, interact_manual


In [ ]:
@interact
def show_articles_more_than(Campo=[str(x) for x in list(range(1990,2020))],x=(0, 1, 0.01)):
    return IDH_final.loc[IDH_final[Campo] > x,['Country',Campo]]


interactive(children=(Dropdown(description='Campo', options=('1990', '1991', '1992', '1993', '1994', '1995', '…

In [67]:
clasificacion_paises = ["High Human Development","Medium Human Development","Medium Human Development","Low Human Development"]

@interact
def show_articles_more_than(Campo=[str(x) for x in list(range(1990,2020))],Clasificacion=clasificacion_paises):
    if Clasificacion =="High Human Development":
        return IDH_final.loc[IDH_final[Campo] > 0.80,['Country',Campo]]
    elif Clasificacion == "Medium Human Development":
        return IDH_final.loc[IDH_final[Campo] > 0.70,['Country',Campo]]
    elif Clasificacion == "Medium Human Development":
        return IDH_final.loc[IDH_final[Campo] > 0.50,['Country',Campo]]
    elif Clasificacion == "Low Human Development":
        return IDH_final.loc[0.50 > IDH_final[Campo],['Country',Campo]]

interactive(children=(Dropdown(description='Campo', options=('1990', '1991', '1992', '1993', '1994', '1995', '…

Ahora se almacenan los resultados en un documento __.csv__.

In [68]:
IDH_final.to_csv('HDI_data.csv', index=False)